In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import pandas as pd
from sqlalchemy import create_engine
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import numpy as np

# -----------------------------
# CONFIGURATION BASE DE DONNÉES
# -----------------------------
DB_USER = "postgres"
DB_PASSWORD = "votre_mdp"
DB_HOST = "yamanote.proxy.rlwy.net"
DB_PORT = "19612"
DB_NAME = "grossi"

engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

# -----------------------------
# FASTAPI
# -----------------------------
app = FastAPI(title="Stock Prediction API")

# -----------------------------
# CHARGER LES DONNÉES
# -----------------------------
def load_stock_data():
    stock_df = pd.read_sql("SELECT * FROM stock", engine)
    tm_df = pd.read_sql("SELECT * FROM typemouvementstock", engine)

    df = stock_df.merge(tm_df, left_on='type_mouvement_id', right_on='id', how='left')

    # Calcul stock net
    df['quantite_net'] = df.apply(
        lambda row: row['quantite'] if row['code'] in ['ENTREE','RETOUR_CLI','AJUST_POS']
                    else -row['quantite'] if row['code'] in ['SORTIE','RETOUR_FOUR','AJUST_NEG']
                    else 0,
        axis=1
    )

    # Somme par produit
    stock_dispo = df.groupby('produit_id')['quantite_net'].sum().reset_index()
    stock_dispo.rename(columns={'quantite_net':'stock_disponible'}, inplace=True)

    # Ajouter features simples pour ML (exemple : jours depuis création)
    df['created_at'] = pd.to_datetime(df['created_at'])
    df['jours_depuis_creation'] = (pd.Timestamp.now() - df['created_at']).dt.days
    stock_dispo = stock_dispo.merge(df[['produit_id','jours_depuis_creation']].drop_duplicates('produit_id'), on='produit_id')
    
    return stock_dispo

# -----------------------------
# ENTRAINER LE MODELE ML
# -----------------------------
def train_model():
    df = load_stock_data()
    X = df[['jours_depuis_creation']].values  # Features simples
    y = df['stock_disponible'].values         # Target

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    return model

model = train_model()

# -----------------------------
# MODELE INPUT
# -----------------------------
class StockPredictRequest(BaseModel):
    produit_id: int
    jours_futurs: int

# -----------------------------
# API ENDPOINTS
# -----------------------------
@app.get("/stocks")
def get_stock_dispo():
    return load_stock_data().to_dict(orient='records')

@app.post("/predict")
def predict_stock(req: StockPredictRequest):
    df = load_stock_data()
    # Récupère jours_depuis_creation du produit
    row = df[df['produit_id']==req.produit_id]
    if row.empty:
        return {"error": "Produit non trouvé"}
    
    jours_depuis_creation = row['jours_depuis_creation'].values[0] + req.jours_futurs
    X_pred = np.array([[jours_depuis_creation]])
    stock_future = model.predict(X_pred)[0]

    return {"produit_id": req.produit_id, "stock_predicted": stock_future}
